In [ ]:
#Import required packages
import cv2 as cv
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2 as cv
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import pandas as pd
from AI_functions import resnet18, BarlowTwins, ImageCollateFunction,BarlowTwinsLoss,CellDataset,data_generator

latent_dim=124
epochs=200

#Insert filepath for local files  FOR THIBAUT
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\BoneMarrow_sample1"
readpath = basepath + "\\Raw_Images"
outpath = basepath + "\\Outputs"
file_prefix="\\sample1_"
maskpath=basepath+"\\ExportedMasks"
image_dim=64 #Dim of the final images
nuclear_channel="Ch7"
cellmask_channel="Ch1_mask"
df=pd.read_csv(outpath+"\\cell_info.csv")
cell_names=df["Cell_ID"].to_numpy()
image_dict={}
for cell_name in cell_names:
    image_dict[cell_name]={}
#Find Channels
names=[]
for entry in os.listdir(outpath): #Read all files
    if os.path.isfile(os.path.join(outpath, entry)):
        if entry!='image_ID.npy':
            names.append(entry)
channels=[name[:-4] for name in names if name[-4:]=='.npy']
print("Channels found:",channels)
data_dict={}
for channel in channels:
    data_dict[channel]=np.load(outpath+"\\"+channel+'.npy')
#Break up array
for channel in data_dict:
    dims=data_dict[channel].shape
    n=dims[0]//image_dim
    l=dims[1]//image_dim
    index=0
    for i in range(n):
        for j in range(l):
            img=data_dict[channel][i*image_dim:i*image_dim+image_dim,j*image_dim:j*image_dim+image_dim]
            image_dict[cell_names[index]][channel]=img
            index+=1

def to_onehot(my_list):
    return_list=[]
    for i,elem in enumerate(my_list):
        j=np.where(np.unique(labels)==elem)
        return_list.append(np.zeros((len(np.unique(my_list)))))
        return_list[-1][j]=1
    return np.array(return_list)
Channels=['Ch1']  #Channel to be fed to the NN

images_with_index = []
for image_i in image_dict:
    image=cv.merge([image_dict[image_i][i] for i in Channels])
    images_with_index.append((int(image_i),image))
    
images=np.array([image[1] for image in images_with_index])
names=np.array([image[0] for image in images_with_index])
labels=df['Cell_Type'].to_numpy()
assert sum(names!=df['Cell_ID'].to_numpy()) ==0  #Check that the order has been preserved
DNA_pos=df["DNA_pos"].to_numpy()
Touches_Boundary=df["Touches_boundary"].to_numpy()
labels=df['Cell_Type'].to_numpy()
idx_to_keep=np.array(DNA_pos==1,dtype=int)+np.array(Touches_Boundary==0,dtype=int)+np.array(labels==0,dtype=int)+np.array(labels==2,dtype=int)==3  #keep dnapos, no touch boundarym APC and Other
#Filter
images=images[idx_to_keep]
names=names[idx_to_keep]
labels=labels[idx_to_keep]
labels=to_onehot(labels)

mini=int(round(abs(np.array(images).min()),0))
images=images+abs(np.array(images).min())
mean=np.array(images).mean()
maxi=np.array(images).max()
std=np.array(images).std()



In [ ]:
returned=data_generator(images,labels,names,mini,train_test_split = 0.8,batch_size = 100)
train_data,train_data1,train_labels,train_ID,test_data,batch_size,mean_loader,std_loader,sampler=returned

In [ ]:
transform_train = transforms.Compose(
    [transforms.ToPILImage(),transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=180,fill=37),
    transforms.ToTensor(),
    transforms.Resize([64, 64]),

   transforms.Normalize(mean=[mean_loader], std=[std_loader]),  # for grayscale images
   ])
   
rotated = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Resize([64, 64]),
    transforms.Normalize(mean=[mean_loader], std=[std_loader]),  # for grayscale images
    transforms.RandomRotation(degrees=180,fill=0),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.Lambda(lambda x: x if 0.5>np.random.rand() else transforms.functional.invert(x)-1),
    transforms.Lambda(lambda x: x *(1+0.2*np.random.randn()) ),
   # transforms.Lambda(lambda x: x + (0.1**0.5)*torch.randn(64, 64) )
    
    ]) 

train_data = CellDataset(train_data1,train_labels,train_ID, rotated)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)#,sampler=sampler)



In [ ]:
my_imgs=next(iter(train_loader))
plt.imshow(my_imgs[0][0][2][0])

plt.show()
plt.imshow(my_imgs[0][1][2][0])

In [ ]:


train_data = CellDataset(train_data1,train_labels,train_ID, rotated)
train_loader = DataLoader(train_data, batch_size=524,sampler=sampler)  #shuffle=True
test_loader = DataLoader(test_data, batch_size=524, shuffle=True)

def get_val():
    total_loss = 0
    model.eval()
    for (x0, x1), _ in test_loader:
        x0 = x0.to(device)
        x1 = x1.to(device)
        z0 = model(x0)
        z1 = model(x1)
        loss = criterion(z0, z1)
        total_loss += loss.detach()

    print("Validation Loss:",int(total_loss / len(test_loader)))
    return int(total_loss / len(test_loader))
    
def train():
    total_loss = 0
    model.train()
    for (x0, x1), _ in train_loader:
        x0 = x0.to(device)
        x1 = x1.to(device)
        z0 = model(x0)
        z1 = model(x1)
        loss = criterion(z0, z1)
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    avg_loss = total_loss / len(train_loader)
    return avg_loss


resnet = resnet18(num_classes=latent_dim,channel_num=len(Channels))
backbone = resnet
model = BarlowTwins(backbone,latent_dim)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


collate_fn = ImageCollateFunction(input_size=32)


criterion = BarlowTwinsLoss(device=device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.06)

print("Starting Training")

train_losses=[]
val_losses=[]
for epoch in range(epochs):
    avg_loss=train()
    print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")
    train_losses.append(avg_loss)
    val_losses.append(get_val())

In [ ]:
train_losses=[int(i.detach().cpu()) for i in train_losses]
plt.figure(figsize=(10,10))
plt.plot(train_losses)
plt.plot(val_losses)
plt.savefig(basepath+"/Results/Loss"+str(train_losses[-1])+".png",bbox_inches="tight")


In [ ]:
train_data = CellDataset(train_data1,train_labels,train_ID,rotated)
train_loader = DataLoader(train_data, batch_size=124, shuffle=True)#,sampler=sampler)

barlow=np.zeros((len(train_data1),latent_dim))
names=np.zeros((len(train_data1)))
labels=np.zeros((len(train_data1)))
i=0
for (x0, x1), Y in train_loader:
    x0 = x0.to(device)
    x1 = x1.to(device)
    latents=model.backbone(x0).detach().cpu().numpy()
    names[i:i+len(latents)]=Y[:,1]
    labels[i:i+len(latents)]=Y[:,0]
    barlow[i:i+len(latents)]=latents
    i+=len(latents)
    print(i,len(train_data1))

df3=pd.DataFrame(barlow)
df3["Cell_ID"]=names
df3.to_csv(basepath+"/Results/Barlow_latents"+str(train_losses[-1])+".csv", index = False, header=True)

    